# Hop ranges, slices, and labels

Small, self-contained examples showing single/variable hops, post-filtered output slices, and optional hop labels on a toy graph.


In [1]:
import pandas as pd
import graphistry

edges = pd.DataFrame({
    's': ['a', 'b1', 'c1', 'd1', 'a', 'b2'],
    'd': ['b1', 'c1', 'd1', 'e1', 'b2', 'c2']
})

nodes = pd.DataFrame({'id': ['a', 'b1', 'c1', 'd1', 'e1', 'b2', 'c2']})

g = graphistry.edges(edges, 's', 'd').nodes(nodes, 'id')
seeds = pd.DataFrame({g._node: ['a']})

print('Edges:')
print(edges)
print()
print('Nodes:')
print(nodes)


Edges:
    s   d
0   a  b1
1  b1  c1
2  c1  d1
3  d1  e1
4   a  b2
5  b2  c2

Nodes:
   id
0   a
1  b1
2  c1
3  d1
4  e1
5  b2
6  c2


In [2]:
# Exactly 1 hop (no labels)
hop_1 = g.hop(seeds, min_hops=1, max_hops=1)
print("1-hop nodes from 'a' (first hop):")
print(hop_1._nodes.sort_values(g._node))
print()
print('1-hop edges:')
print(hop_1._edges.sort_values([g._source, g._destination]))


1-hop nodes from 'a' (first hop):
   id
0   a
1  b1
2  b2

1-hop edges:
   s   d
0  a  b1
1  a  b2


In [3]:
# Exactly 3 hops (no labels, default keeps path)
hop_3 = g.hop(seeds, min_hops=3, max_hops=3)
print("3-hop nodes from 'a' (path to third hop):")
print(hop_3._nodes.sort_values(g._node))
print()
print('3-hop edges:')
print(hop_3._edges.sort_values([g._source, g._destination]))


3-hop nodes from 'a' (path to third hop):
   id
0   a
1  b1
4  b2
2  c1
5  c2
3  d1

3-hop edges:
    s   d
0   a  b1
3   a  b2
1  b1  c1
4  b2  c2
2  c1  d1


In [4]:
# Range 1..3 hops (combined)
hop_range = g.hop(seeds, min_hops=1, max_hops=3)
print("Nodes within 1..3 hops from 'a':")
print(hop_range._nodes.sort_values(g._node))
print()
print('Edges within 1..3 hops:')
print(hop_range._edges.sort_values([g._source, g._destination]))


Nodes within 1..3 hops from 'a':
   id
0   a
1  b1
4  b2
2  c1
5  c2
3  d1

Edges within 1..3 hops:
    s   d
0   a  b1
3   a  b2
1  b1  c1
4  b2  c2
2  c1  d1


In [5]:
# Traverse 2..4 hops; output slice 3..4 with hop labels
hop_slice = g.hop(
    seeds,
    min_hops=2,
    max_hops=4,
    output_min_hops=3,
    output_max_hops=4,
    label_node_hops='hop',
    label_edge_hops='edge_hop'
)
print('Nodes in hops 3..4 (after traversing 2..4):')
print(hop_slice._nodes.sort_values(['hop', g._node]))
print()
print('Edges in hops 3..4:')
print(hop_slice._edges.sort_values(['edge_hop', g._source, g._destination]))


Nodes in hops 3..4 (after traversing 2..4):
   id  hop
0  d1    3
1  e1    4

Edges in hops 3..4:
    s   d  edge_hop
0  c1  d1         3
1  d1  e1         4


In [6]:
# Output slice below traversal min (keeps earlier hops)
hop_slice_below = g.hop(
    seeds,
    min_hops=3,
    max_hops=3,
    output_min_hops=1,
    label_node_hops='hop',
    label_edge_hops='edge_hop',
    label_seeds=True
)
print('Nodes when output_min < min_hops (labels on, seeds labeled):')
print(hop_slice_below._nodes.sort_values(['hop', g._node]))
print()
print('Edges when output_min < min_hops:')
print(hop_slice_below._edges.sort_values(['edge_hop', g._source, g._destination]))


Nodes when output_min < min_hops (labels on, seeds labeled):
   id  hop
0   a    0
1  b1    1
4  b2    1
2  c1    2
5  c2    2
3  d1    3

Edges when output_min < min_hops:
    s   d  edge_hop
0   a  b1         1
3   a  b2         1
1  b1  c1         2
4  b2  c2         2
2  c1  d1         3


In [7]:
# Output slice max above traversal max (allowed, capped by traversal)
hop_slice_above = g.hop(
    seeds,
    min_hops=2,
    max_hops=2,
    output_max_hops=5,
    label_edge_hops='edge_hop'
)
print('Edges when output_max > traversal max (still capped at traversal):')
print(hop_slice_above._edges.sort_values(['edge_hop', g._source, g._destination]))


Edges when output_max > traversal max (still capped at traversal):
    s   d  edge_hop
0   a  b1         1
2   a  b2         1
1  b1  c1         2
3  b2  c2         2


In [8]:
# Invalid output slice (output_min > max_hops)
try:
    g.hop(seeds, min_hops=2, max_hops=3, output_min_hops=5)
except Exception as e:
    print('Invalid output_min > max_hops:', e)

# Invalid output slice (output_max < min_hops)
try:
    g.hop(seeds, min_hops=2, max_hops=3, output_max_hops=1)
except Exception as e:
    print('Invalid output_max < min_hops:', e)


Invalid output_min > max_hops: output_min_hops (5) cannot exceed max_hops traversal bound (3)
Invalid output_max < min_hops: output_max_hops (1) cannot be below min_hops traversal bound (2)


In [9]:
# Hop labels without seed labels
hop_labels_off = g.hop(
    seeds,
    min_hops=1,
    max_hops=3,
    label_node_hops='hop',
    label_edge_hops='edge_hop',
    label_seeds=False
)
print('Nodes with hop labels (seeds not labeled):')
print(hop_labels_off._nodes.sort_values(['hop', g._node]))
print()
print('Edges with hop labels (seeds not labeled):')
print(hop_labels_off._edges.sort_values(['edge_hop', g._source, g._destination]))


Nodes with hop labels (seeds not labeled):
   id  hop
0  b1    1
3  b2    1
1  c1    2
4  c2    2
2  d1    3

Edges with hop labels (seeds not labeled):
    s   d  edge_hop
0   a  b1         1
3   a  b2         1
1  b1  c1         2
4  b2  c2         2
2  c1  d1         3


In [10]:
# Hop labels with seed labels
hop_labels_on = g.hop(
    seeds,
    min_hops=1,
    max_hops=3,
    label_node_hops='hop',
    label_edge_hops='edge_hop',
    label_seeds=True
)
print('Nodes with hop labels (seeds labeled hop=0):')
print(hop_labels_on._nodes.sort_values(['hop', g._node]))
print()
print('Edges with hop labels (seeds labeled hop=0):')
print(hop_labels_on._edges.sort_values(['edge_hop', g._source, g._destination]))


Nodes with hop labels (seeds labeled hop=0):
   id  hop
0   a    0
1  b1    1
4  b2    1
2  c1    2
5  c2    2
3  d1    3

Edges with hop labels (seeds labeled hop=0):
    s   d  edge_hop
0   a  b1         1
3   a  b2         1
1  b1  c1         2
4  b2  c2         2
2  c1  d1         3
